# DL building

In [2]:
%r
library(boot)
library(dplyr, warn.conflicts=FALSE)
library(ggplot2)
library(sparklyr, warn.conflicts=FALSE)
library(tidyr, warn.conflicts=FALSE)

sc = spark_connect(method="databricks")

# Save result tables so we can access them from hala


In [3]:
from moztelemetry.dataset import Dataset
import pandas as pd
from pyspark.sql import Row
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, DoubleType, LongType, MapType
from statsmodels.stats.weightstats import DescrStatsW

In [4]:
  Dataset.from_source("telemetry").schema

Out[ 3 ]: 
[u'submissionDate',
 u'sourceName',
 u'sourceVersion',
 u'docType',
 u'appName',
 u'appUpdateChannel',
 u'appVersion',
 u'appBuildId']

In [5]:


pings = (
  Dataset
  .from_source("telemetry")
  .where(
    docType="main",
  )
  .where(appUpdateChannel="beta")
  .where(submissionDate='20181220')
  .select(
    app_build_id="application.buildId",
    environment='environment.system',
    displaylist_time="payload.processes.content.histograms.PAINT_BUILD_DISPLAYLIST_TIME",
    client_id="clientId",
    normalized_channel="meta.normalizedChannel",
    profile_subsession_counter="payload.info.profileSubsessionCounter",
    session_id="payload.info.sessionId",
  )
  .records(sc, sample=0.1)
)

WARNING: THIS IS NOT A REPRESENTATIVE SAMPLE.
This 'sampling' is based on s3 files and is highly
susceptible to skew. Use only for quicker performance
while prototyping.
fetching 17368.29883MB in 9745 files...

In [6]:
pings.count()

Out[ 6 ]: 616532

In [7]:
pings.take(1)

Out[ 7 ]: 
[{'app_build_id': u'20181217180946',
 'client_id': u'915b1712-ace8-46c7-b1ff-ab5defafe439',
 'displaylist_time': {u'bucket_count': 50,
 u'histogram_type': 0,
 u'range': [1, 1000],
 u'sum': 17576,
 u'values': {u'0': 417,
 u'1': 962,
 u'10': 53,
 u'11': 51,
 u'12': 46,
 u'14': 7,
 u'16': 1,
 u'2': 1933,
 u'23': 1,
 u'26': 0,
 u'3': 864,
 u'4': 627,
 u'5': 427,
 u'6': 206,
 u'7': 87,
 u'8': 126,
 u'9': 96}},
 'environment': {u'appleModelId': None,
 u'cpu': {u'cores': 2,
 u'count': 4,
 u'extensions': [u'hasMMX',
 u'hasSSE',
 u'hasSSE2',
 u'hasSSE3',
 u'hasSSSE3',
 u'hasSSE4_1',
 u'hasSSE4_2',
 u'hasAVX'],
 u'family': 6,
 u'l2cacheKB': 256,
 u'l3cacheKB': 3072,
 u'model': 42,
 u'speedMHz': 2294,
 u'stepping': 7,
 u'vendor': u'GenuineIntel'},
 u'gfx': {u'ContentBackend': u'Skia',
 u'D2DEnabled': False,
 u'DWriteEnabled': True,
 u'adapters': [{u'GPUActive': True,
 u'RAM': None,
 u'description': u'Intel(R) HD Graphics Family',
 u'deviceID': u'0x0116',
 u'driver': u'igdumd64 igd10umd64 igd10umd64 igdumdx32 igd10umd32 igd10umd32',
 u'driverDate': u'9-30-2011',
 u'driverVersion': u'8.882.2.3000',
 u'subsysID': u'185d103c',
 u'vendorID': u'0x8086'}],
 u'features': {u'compositor': u'basic',
 u'd2d': {u'status': u'unavailable', u'version': u'1.1'},
 u'd3d11': {u'status': u'blacklisted'},
 u'gpuProcess': {u'status': u'blacklisted'},
 u'webrender': {u'status': u'unavailable'},
 u'wrQualified': {u'status': u'blocked'}},
 u'monitors': [{u'pseudoDisplay': False,
 u'refreshRate': 60,
 u'screenHeight': 768,
 u'screenWidth': 1366}]},
 u'hdd': {u'binary': {u'model': u'TOSHIBA MK5076GSX', u'revision': u'GS00'},
 u'profile': {u'model': u'TOSHIBA MK5076GSX', u'revision': u'GS00'},
 u'system': {u'model': u'TOSHIBA MK5076GSX', u'revision': u'GS00'}},
 u'isWow64': False,
 u'memoryMB': 4044,
 u'os': {u'installYear': 2016,
 u'locale': u'en-US',
 u'name': u'Windows_NT',
 u'servicePackMajor': 1,
 u'servicePackMinor': 0,
 u'version': u'6.1',
 u'windowsBuildNumber': 7601},
 u'virtualMaxMB': 8388608},
 'normalized_channel': u'beta',
 'profile_subsession_counter': 1418,
 'session_id': u'be135391-72bc-4939-b64a-8008e32b2f5e'}]